# Card Shuffling

Deck with $n$ cards.

**Random-to-Random Shuffle**:
 1. Remove a card u.a.r. from the deck
 1. Insert it u.a.r.

One shuffle acts on the deck by one of the following elements of $S_n$:
$$S=\{(1)\}\cup\{(a\ a+1)\ :\ a\in [n-1]\}\cup\{(a\ a+1\ \cdots b)^\pm\ :\ a, b\in [n], |a-b| > 1\} \subset S_n.$$

Note: $|S|=1+(n-1)+2n(n-3)$ ?

For any $x\in S_n$, and $s\in S$, $\mathbb{P}(x, xs)=\begin{cases}\frac{1}{n} & s=(1)\\ \frac{2}{n^2} & s= (a\ a+1)\\ \frac{1}{n^2}&s=(a\ a+1\ \cdots\ b)\end{cases}$

**Modified Random-to-Random Shuffle**
 1. Remove a card u.a.r. from the deck
 1. Insert it u.a.r. *among all positions, excluding the one below it or the top position if the bottom card is selected*
 
For any $x\in S_n$, $\mathbb{P}(x, xs)=\begin{cases}\frac{1}{n} & s=(1)\\ \frac{1-\frac{1}{n}}{|S|-1} & s\in S\setminus\{(1)\}\end{cases}$

**Goal**

Let $T$ be the first time that all cards have been picked ($T\geq n$).

We want to compute $\mathbb{P}(X_t=\sigma\mid T\leq t)$ for all $\sigma\in S_n$ and $t$.

___

In [6]:
import random as r

In [228]:
def get_deck(n):
    return list(range(1, n + 1))

def rtr(deck:list):
    n = len(deck)
    card = r.choice(deck)
    new_deck = deck.copy()
    new_deck.remove(card)
    new_loc = r.randint(0, n - 1)
    new_deck.insert(b, card)
    return new_deck, card

def m_rtr(deck:list):
    n = len(deck)
    a = r.randint(0, n - 1)
    card = deck[a]
    new_deck = deck.copy()
    new_deck.remove(card)
    b = r.choice([pos % n for pos in range(a + 2, a + n + 1)])
    new_deck.insert(b, card)
    return new_deck, card

In [148]:
def T_trial(shuffle, n):
    deck = get_deck(n)
    marked = set()
    t = 0
    while len(marked) < n:
        deck, card = shuffle(deck)
        marked.add(card)
        t += 1
    return t

____

In [279]:
class Node:
    def __init__(self, deck:list, marked:set=set(), prob:float=1, depth:int=0):
        self.deck = deck
        self.marked = marked
        self.children = []
        self.prob = prob # probability that this node is reached from its parent
        self.depth = depth
        
    def set_deck(self, deck):
        self.deck = deck
    
    def set_marked(self, marked):
        self.marked = marked
        
    def iterate_children(self):
        for child in self.children:
            yield child
        return None
    
    def __str__(self):
        return "\n".join([
            "_" * (len(self.deck)*2 + 1),
            "|" + "|".join([" ", "*"][card in self.marked] for card in self.deck) + "|",
            "|" + "|".join(str(card) for card in self.deck) + "|" + f"   prob: {self.prob}",
            "‾" * (len(self.deck)*2 + 1),
        ])
    
    def __repr__(self):
        return str(self)
    
    def generate_children(self):
        """
        make every possible move
        create a new child node for each move
        link in children list
        """
        
        n = len(self.deck)
        
        for position, card in enumerate(self.deck):
            for pre_new_loc in range(position + 2, position + 1 + n):
                new_deck = self.deck.copy()
                new_deck.remove(card)
                new_loc = pre_new_loc % n
                new_deck.insert(new_loc, card)
                new_marked = self.marked.copy()
                new_marked.add(card)
                node = Node(new_deck, new_marked, 1 / (n * (n-1)), self.depth + 1)
                self.children.append(node)

In [269]:
class Tree:
    def __init__(self, n):
        self.n = n
        self.top = Node(get_deck(n))
    
    def __str__(self):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            for child in node.children:
                print()
    
    def P(t:int, sigma:list):
        if t < self.n:
            raise BaseException(f"t must be at least n={self.n}")
        if sorted(sigma) != get_deck(n):
            raise BaseException("sigma must be a permutation of n cards")
        
        while 